In [4]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('homework_1.1.csv')

In [3]:
df

,Unnamed: 0,X1,X2,X3,Y
0,0,-0.440646,-0.390227,0.156718,-0.877671
1,1,-3.810099,-1.304665,-1.105117,-10.130388
2,2,-1.425451,-0.340049,1.115908,0.284068
3,3,-1.325750,0.161906,-0.254670,-1.994344
4,4,3.120263,1.487343,-1.164839,2.030030
...,...,...,...,...,...
995,995,1.281625,-0.062024,-0.109231,1.206058
996,996,-3.047558,-1.121675,0.678439,-3.229262
997,997,-0.028757,-0.191722,1.191790,3.160897
998,998,0.415266,0.373086,1.172636,4.033302


In [ ]:
X_values = df.drop(columns=['Y'])
y_values = df['Y']

model = LinearRegression()
model.fit(X_values, y_values)

predictions = model.predict(X_values)



In [14]:
X_with_intercept = np.column_stack((np.ones(X_values.shape[0]), X_values))

# === Step 4: Fit model manually ===
XtX = X_with_intercept.T @ X_with_intercept
XtX_inv = np.linalg.inv(XtX)
beta_hat = XtX_inv @ X_with_intercept.T @ y_values

# === Step 5: Compute residuals, error variance (sigma^2) ===
y_hat = X_with_intercept @ beta_hat
residuals = y_values - y_hat
n, p = X_with_intercept.shape
sigma_squared = (residuals.T @ residuals) / (n - p)

# === Step 6: Compute standard errors and t-statistics ===
var_beta_hat = sigma_squared * XtX_inv
standard_errors = np.sqrt(np.diag(var_beta_hat))
t_stats = beta_hat / standard_errors

# === Step 7: Match to column names ===
feature_names = df.columns[:-1].tolist()
feature_names_with_intercept = ['intercept'] + feature_names

# === Step 8: Display results ===
print(f"{'Variable':<15}{'Coefficient':>15}{'T-Statistic':>15}")
print("-" * 45)
for name, coef, t in zip(feature_names_with_intercept, beta_hat, t_stats):
    print(f"{name:<15}{coef:>15.4f}{t:>15.4f}")

Variable           Coefficient    T-Statistic
---------------------------------------------
intercept               0.0476         1.4959
Unnamed: 0             -0.0001        -1.6310
X1                      1.0092        60.9782
X2                      1.9599        53.0416
X3                      2.9762       196.7776


In [22]:


# Prepare predictors (X) and target (y)
X = df.iloc[:, :-1].values  # All columns except the last are predictors
y = df.iloc[:, -1].values   # Last column is the target variable

# Fit multiple regression model (all X's)
model_all = LinearRegression().fit(X, y)
coeff_all = model_all.coef_

# Initialize list to store the differences in coefficients
differences = []

# For each feature, fit univariate model and calculate difference
for i in range(X.shape[1]):
    X_i = X[:, i].reshape(-1, 1)  # Reshape X_i as a 2D array
    coeff_single = LinearRegression().fit(X_i, y).coef_[0]
    differences.append(np.abs(coeff_all[i] - coeff_single))

# Find the feature with the greatest difference
max_diff_index = np.argmax(differences)
most_affected_feature = df.columns[max_diff_index]

# Output the result
print(f"Feature with the greatest difference: {most_affected_feature}")
print(f"Difference: {differences[max_diff_index]:.4f}")

Feature with the greatest difference: X2
Difference: 2.1237


In [24]:
df_four_five = pd.read_csv('/Users/emmastiller/2025-summer-mod-6/homework_1.2.csv')
from sklearn.neighbors import NearestNeighbors

In [34]:
z_values = df_four_five[['Z']].values
neighbors = NearestNeighbors(n_neighbors=2, metric='euclidean')
neighbors.fit(z_values)

distances, indices = neighbors.kneighbors(z_values)
df_four_five['farthest_match_index'] = indices[:,1]
df_four_five['distance_to_farthest_match'] = distances[:,1]
max_distance = (df_four_five['distance_to_farthest_match']).max()
print(df_four_five[['Z', 'closest_match_index', 'distance_to_closest_match']])
print(max_distance)

           Z  closest_match_index  distance_to_closest_match
0   0.548814                    3                   0.003930
1   0.715189                   93                   0.001138
2   0.602763                   73                   0.002082
3   0.544883                    0                   0.003930
4   0.423655                   29                   0.008993
..       ...                  ...                        ...
95  0.183191                   64                   0.013391
96  0.586513                   88                   0.010566
97  0.020108                   16                   0.000111
98  0.828940                   17                   0.003680
99  0.004695                   34                   0.014094

[100 rows x 3 columns]
0.022936375279736132


In [35]:
df_x0 = df_four_five[df_four_five['X'] == 0]  # Data where X = 0
df_x1 = df_four_five[df_four_five['X'] == 1]  # Data where X = 1

# Step 2: Extract Z values for both subsets (X = 0 and X = 1)
Z_x0 = df_x0[['Z']].values  # Z values for X = 0
Z_x1 = df_x1[['Z']].values  # Z values for X = 1

# Step 3: Initialize NearestNeighbors and fit on Z values for X = 0
neighbors = NearestNeighbors(n_neighbors=1, metric='euclidean')  # We need the closest neighbor
neighbors.fit(Z_x0)  # Fit the model on Z values of X = 0

# Step 4: Find the closest match for each X = 1 item in the X = 0 set
distances, indices = neighbors.kneighbors(Z_x1)  # For each Z in X = 1, find the closest Z in X = 0

# Step 5: Add the distance to the farthest match
df_x1['distance_to_closest_match'] = distances.flatten()  # Flatten to get a 1D array

# Step 6: Calculate the maximum distance among all the closest matches
max_distance = df_x1['distance_to_closest_match'].max()

# Output the maximum distance to the farthest match
print(f"The distance of the farthest match is: {max_distance}")

The distance of the farthest match is: 0.2102170871093757


/var/folders/cm/9bmjknls3816ptb8qwtfw9940000gn/T/ipykernel_1866/4085741063.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x1['distance_to_closest_match'] = distances.flatten()  # Flatten to get a 1D array


In [38]:
distances, indices = neighbors.radius_neighbors(Z_x1)
best_matches_x0 = []  # Flatten to a 1D array to easily compute averages

for i in range(len(Z_x1)):
    if len(indices[i]) > 0:
        # Take the closest match (first in the list of indices)
        best_matches_x0.append(Z_x0[indices[i][0], 0])
    else:
        # If no match is found, append NaN or handle as needed
        best_matches_x0.append(float('nan'))

# Convert the list of best matches into a Pandas series for easy calculations
best_matches_x0 = pd.Series(best_matches_x0)

# Step 6: Compute the average Z for X = 0
average_x0 = df_x0['Z'].mean()

# Step 7: Compute the average Z for X = 1 with the best matches from X = 0
average_best_match_x1 = best_matches_x0.mean()

# Step 8: Calculate the effect (difference between averages)
effect = average_best_match_x1 - average_x0

print(effect)

0.21706886288465266


In [40]:
df_x0 = df_four_five[df_four_five['X'] == 0]  # Data where X = 0
df_x1 = df_four_five[df_four_five['X'] == 1]  # Data where X = 1

# Step 2: Extract Z values for both subsets (X = 0 and X = 1)
Z_x0 = df_x0[['Z']].values  # Z values for X = 0
Z_x1 = df_x1[['Z']].values  # Z values for X = 1

# Step 3: Initialize NearestNeighbors with radius_neighbors (find matches within a distance of 0.2)
neighbors = NearestNeighbors(radius=0.2, metric='euclidean')  # Radius of 0.2
neighbors.fit(Z_x0)  # Fit the model on Z values of X = 0

# Step 4: Find all matches within 0.2 distance for each X = 1 item in the X = 0 set
distances, indices = neighbors.radius_neighbors(Z_x1)  # For each Z in X = 1, find all Z in X = 0 within the radius

# Step 5: For each X = 1, get all the corresponding matches from X = 0
matches_x0 = []

for i in range(len(Z_x1)):
    if len(indices[i]) > 0:
        # Append all matched Z values from X = 0 for each X = 1
        matches_x0.extend(Z_x0[indices[i], 0])  # Extracting the Z values for X = 0 samples

# Convert the list of all matched Z values into a Pandas series for easy calculations
matches_x0_series = pd.Series(matches_x0)

# Step 6: Compute the average Z for X = 0
average_x0 = df_x0['Z'].mean()

# Step 7: Compute the average Z for X = 1 with all matches from X = 0 within distance 0.2
average_matches_x1 = matches_x0_series.mean()

# Step 8: Calculate the effect (difference between averages)
effect = average_matches_x1 - average_x0
print(effect)

0.10828131989901935


In [41]:
duplicate_count = 0

for i in range(len(Z_x1)):
    # indices[i] gives us the list of matching indices for the i-th X=1 sample
    if len(indices[i]) > 1:
        # We exclude the first match (the first one is not a duplicate)
        duplicate_count += len(indices[i]) - 1  # Count all but the first match
print({duplicate_count})

{691}


## Week 2 Quiz 

In [1]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
df_2 = pd.read_csv('homework_2.1.csv')

In [3]:
print(df_2.columns)

Index(['group', 'time', 'G1', 'G2', 'G3'], dtype='object')


## Problem 2.1

In [4]:
group_col = ['group']
outcome = ['G1']

In [5]:
df_2['post_treatment'] = df_2['group']* df_2['time']

In [6]:
results = sm.OLS(df_2.G1, sm.add_constant(df_2[["time"]])).fit()
results.params

const    0.104236
time     0.008498
dtype: float64

In [7]:
# model = smf.ols('G1 ~ C(group) + C(time) + post_treatment', data=df_2).fit()
# print(model.summary())

In [ ]:
# Answer 2.1 = C(group)[T.1] = -0.3360

## Problem 2.2

In [ ]:
# make new dataframe with time, G - concat

In [8]:
## common linear coefficient 
# predict any G from time * x 

results = sm.OLS(df_2.G, sm.add_constant(df_2[["time"]])).fit()
results.params


AttributeError: 'DataFrame' object has no attribute 'G'

In [31]:
# Answer 2.2 = Intercept = 0.8820

## Problem 2.3 

In [9]:
## do (Y when x=1) - (Y when X=0)
# to get treated vs untreated data


In [11]:
df_22 = pd.read_csv('homework_2.2.csv')

In [41]:
results = sm.OLS(df_22.Y, sm.add_constant(df_22[["X", "Z"]])).fit()
results.params

const    4.943624
X        2.818696
Z        2.124724
dtype: float64

## Problem 2.4

In [12]:
## do bootstrap sampling 
n_boostrap = 10000
boot_coefs = []

for i in range(n_boostrap): 
    boot_sample = df_22.sample(frac=1, replace=True)
    X0 = boot_sample[boot_sample["X"]==0].copy()
    X1 = boot_sample[boot_sample["X"]==1].copy()
    means = (X1['Y'].mean() - X0['Y'].mean())
    boot_coefs.append(means)
boot_coefs = np.array(boot_coefs)

variance = np.var(boot_coefs)

In [13]:
variance

np.float64(0.03196932133715095)

## Problem 2.5

In [15]:
from scipy.stats import skew

In [16]:
coef_skewness = skew(boot_coefs)

In [17]:
coef_skewness

np.float64(0.058858855211658165)